In [ ]:
BoW+LR

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the UCI Diabetes dataset
#data = pd.read_csv("diabetic_data.csv", usecols=["readmitted", "diag_1", "diag_2", "diag_3"])

# Replace missing values with the string "missing"
data.fillna("missing", inplace=True)

# Create a bag-of-words representation of the diagnosis codes
vectorizer = CountVectorizer(token_pattern="[0-9]+")
X = vectorizer.fit_transform(data[["diag_1", "diag_2", "diag_3"]].apply(lambda x: " ".join(x), axis=1))

# Use logistic regression to predict unplanned readmissions
lr = LogisticRegression()
lr.fit(X, data["readmitted"])

# Calculate accuracy on the training data
y_pred = lr.predict(X)
accuracy = accuracy_score(data["readmitted"], y_pred)
print("Accuracy: {:.3f}".format(accuracy))


Accuracy: 0.552


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
Deepr (Rand-init)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('diabetic_data.csv')
columns_to_remove = ['encounter_id', 'patient_nbr', 'weight', 'payer_code', 'medical_specialty']
data = data.drop(columns_to_remove, axis=1)
# Preprocess the data
data = data.replace('?', np.nan)
data = data.dropna()

# Convert categorical features to numerical using label encoding
cat_cols = data.select_dtypes(include=['object']).columns
label_encoder = LabelEncoder()
for col in cat_cols:
    data[col] = label_encoder.fit_transform(data[col])

# Split into X and y
X = data.drop('readmitted', axis=1)
y = data['readmitted']
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Scale the numerical features
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

# Define the model
model = Sequential()
model.add(Dense(64, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
history = model.fit(X_train, to_categorical(y_train), epochs=50, batch_size=128, 
                    validation_data=(X_test, to_categorical(y_test)), callbacks=[early_stopping])

y_pred = model.predict(X_test)
y_pred_class = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_class)
precision = precision_score(y_test, y_pred_class, average='weighted')
recall = recall_score(y_test, y_pred_class, average='weighted')
confusion = confusion_matrix(y_test, y_pred_class)
print('Accuracy:', accuracy)


<ipython-input-2-eaec9abebf37>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = label_encoder.fit_transform(data[col])


Epoch 1/50
613/613 [==============================] - 3s 3ms/step - loss: 0.9424 - accuracy: 0.5444 - val_loss: 0.9101 - val_accuracy: 0.5638
Epoch 2/50
613/613 [==============================] - 2s 3ms/step - loss: 0.9157 - accuracy: 0.5651 - val_loss: 0.9050 - val_accuracy: 0.5678
Epoch 3/50
613/613 [==============================] - 2s 3ms/step - loss: 0.9093 - accuracy: 0.5690 - val_loss: 0.9033 - val_accuracy: 0.5691
Epoch 4/50
613/613 [==============================] - 2s 4ms/step - loss: 0.9054 - accuracy: 0.5709 - val_loss: 0.9007 - val_accuracy: 0.5723
Epoch 5/50
613/613 [==============================] - 2s 4ms/step - loss: 0.9029 - accuracy: 0.5734 - val_loss: 0.9008 - val_accuracy: 0.5724
Epoch 6/50
613/613 [==============================] - 2s 3ms/step - loss: 0.9010 - accuracy: 0.5749 - val_loss: 0.8996 - val_accuracy: 0.5730
Epoch 7/50
613/613 [==============================] - 2s 3ms/step - loss: 0.9004 - accuracy: 0.5750 - val_loss: 0.8987 - val_accuracy: 0.5721
Epoch 

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
df = pd.read_csv("diabetic_data.csv")

# Preprocess the dataset
df = df[df["readmitted"].isin(["<30", ">30"])]
df["readmitted"] = (df["readmitted"] == "<30").astype(int)
df = df[["diag_1", "diag_2", "diag_3", "readmitted"]]
df = df.dropna()
df["diagnoses"] = df["diag_1"] + " " + df["diag_2"] + " " + df["diag_3"]
df = df[["diagnoses", "readmitted"]]

# Split the dataset into training, validation, and test sets
train_df = df.sample(frac=0.7, random_state=42)
val_df = df.drop(train_df.index).sample(frac=0.5, random_state=42)
test_df = df.drop(train_df.index).drop(val_df.index)

# Train a Word2Vec model on the diagnoses
sentences = [s.split() for s in train_df["diagnoses"].values]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

# Define the tokenizer and vocabulary size
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df["diagnoses"].values)
vocab_size = len(tokenizer.word_index) + 1

# Define the maximum sequence length
max_seq_len = max([len(s.split()) for s in train_df["diagnoses"].values])

# Define the embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv.key_to_index:
      embedding_matrix[i] = w2v_model.wv[word]

# Convert the sentences to sequences of word indices
train_sequences = tokenizer.texts_to_sequences(train_df["diagnoses"].values)
val_sequences = tokenizer.texts_to_sequences(val_df["diagnoses"].values)
test_sequences = tokenizer.texts_to_sequences(test_df["diagnoses"].values)

# Pad the sequences to have the same length
train_sentences = pad_sequences(train_sequences, maxlen=max_seq_len, padding="post", truncating="post")
val_sentences = pad_sequences(val_sequences, maxlen=max_seq_len, padding="post", truncating="post")
test_sentences = pad_sequences(test_sequences, maxlen=max_seq_len, padding="post", truncating="post")

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, embedding_dim, weights=[embedding_matrix], input_length=max_seq_len, trainable=False),
    tf.keras.layers.Conv1D(64, 3, activation="relu", padding="same"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
# Train the model
batch_size = 32
epochs = 10
model.fit(train_sentences, train_df["readmitted"], batch_size=batch_size, epochs=epochs, validation_data=(val_sentences, val_df["readmitted"]))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_sentences, test_df["readmitted"], verbose=2)
print(f"Test accuracy: {test_acc}")

Epoch 1/10
1026/1026 [==============================] - 7s 4ms/step - loss: 0.5635 - accuracy: 0.7564 - val_loss: 0.5508 - val_accuracy: 0.7609
Epoch 2/10
1026/1026 [==============================] - 4s 3ms/step - loss: 0.5565 - accuracy: 0.7577 - val_loss: 0.5500 - val_accuracy: 0.7609
Epoch 3/10
1026/1026 [==============================] - 4s 3ms/step - loss: 0.5551 - accuracy: 0.7576 - val_loss: 0.5499 - val_accuracy: 0.7609
Epoch 4/10
1026/1026 [==============================] - 5s 4ms/step - loss: 0.5543 - accuracy: 0.7575 - val_loss: 0.5489 - val_accuracy: 0.7609
Epoch 5/10
1026/1026 [==============================] - 4s 4ms/step - loss: 0.5546 - accuracy: 0.7576 - val_loss: 0.5492 - val_accuracy: 0.7609
Epoch 6/10
1026/1026 [==============================] - 4s 4ms/step - loss: 0.5543 - accuracy: 0.7577 - val_loss: 0.5489 - val_accuracy: 0.7609
Epoch 7/10
1026/1026 [==============================] - 4s 4ms/step - loss: 0.5538 - accuracy: 0.7575 - val_loss: 0.5490 - val_accuracy: